In [ ]:
cd ./drive/MyDrive/HackathonCode/

/content/drive/MyDrive/HackathonCode


In [ ]:
ls

Deepfake_detection.ipynb               requirements.txt
dlib-19.22.99-cp39-cp39-win_amd64.whl  resnetinceptionv1_epoch_32.pth


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!pip install imageio

In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
import imageio
import dlib
warnings.filterwarnings("ignore")

In [ ]:
def detect_face(frame, detector):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = detector(gray)

    # Check if any faces are detected
    return len(faces) > 0

def save_frames_with_faces_interval(video_path, output_list, interval_ms=20, start_timestamp=1000):
    # Open the video file
    capture = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not capture.isOpened():
        print("Error: Could not open the video file.")
        return

    # Create the Dlib face detection model
    detector = dlib.get_frontal_face_detector()

    # Set the video capture position to the specified timestamp (in milliseconds)
    capture.set(cv2.CAP_PROP_POS_MSEC, start_timestamp)

    # Initialize the timer and frame count
    timer = 0
    frame_count = 0

    # Read and save frames with faces at the specified interval until 10 frames are captured
    while frame_count < 10:
        ret, frame = capture.read()

        if not ret:
            print("End of video.")
            break

        # Check if a face is detected in the frame
        if detect_face(frame, detector):
            # Check if the interval has elapsed
            if timer >= interval_ms:
                output_list.append(frame)
                frame_count += 1  # Increment the frame count
                timer = 0  # Reset the timer
            else:
                timer += 1  # Increment the timer
        else:
            timer = 0  # Reset the timer if no face is detected

    # Release the video capture object
    capture.release()

# Example usage:
#video_path = "./vid.mp4"
frames_with_faces_list = []

#save_frames_with_faces_interval(video_path, frames_with_faces_list, interval_ms=20, start_timestamp=1000)


In [ ]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).to(DEVICE).eval()

In [ ]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [ ]:
def predict(input_data, stupid_data):
    """Predict the label of the input data (image or video frame)"""
    try:
        if isinstance(input_data, np.ndarray):
            # Case: Image
            face = mtcnn(input_data)
        else:
            # Case: Video
            video_reader = imageio.get_reader(str(stupid_data))
            print(str(stupid_data))
            frame = video_reader.get_data(0)  # Read the first frame for processing
            input_image = Image.fromarray(frame)
            face = mtcnn(input_image)

        if face is None:
            raise Exception('No face detected')

        face = face.unsqueeze(0)  # add the batch dimension
        face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

        # convert the face into a numpy array to be able to plot it
        prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
        prev_face = prev_face.astype('uint8')

        face = face.to(DEVICE)
        face = face.to(torch.float32)
        face = face / 255.0
        face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

        target_layers = [model.block8.branch1[-1]]
        use_cuda = True if torch.cuda.is_available() else False
        cam = GradCAM(model=model, target_layers=target_layers, use_cuda=use_cuda)
        targets = [ClassifierOutputTarget(0)]

        grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
        face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

        with torch.no_grad():
            output = torch.sigmoid(model(face).squeeze(0))
            prediction = "real" if output.item() < 0.5 else "fake"

            real_prediction = 1 - output.item()
            fake_prediction = output.item()

            confidences = {
                'real': real_prediction,
                'fake': fake_prediction
            }

        return confidences, face_with_mask

    except Exception as e:
        return "No face detected.", None

In [11]:
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.inputs.Image(label="Input Image", type="numpy"),
        gr.inputs.Video(label = "Input Video")
    ],
    outputs=[
        gr.outputs.Label(label="Result"),
        gr.outputs.Image(label="Face with Explainability", type="pil")
    ],
    title="DeepFake Dectector",
    description="Please enter your image:",
   # logo="Logo.png",
    #thumbnail="Logo.png"
).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>